In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
import seaborn as sns

import matplotlib.pyplot as plt



In [3]:
dataset=pd.read_csv('../../data/BO2_forecast_next_call/forecast_next_call.csv')

In [4]:
dataset.head()

,NOMDPO,LIBPRD,NUMCPT,MOIS,CODGVR,CODLOC,SUM(MNTHT),SUM(QTEPRD),prixHT
0,GOULETTE,GASOIL,AV.BOUGATFA,18-Feb,TUNIS,LE BARDO,28077.686,26.0,1079.911
1,BIZERTE CARBURANT,GASOIL,M. BOURGUIBA,18-Jan,BIZERTE,MENZEL BOURGUIBA,82073.236,76.0,1079.911
2,GOULETTE,SSP SUPER SANS PLOMB,KALAA EL KEBIRA,18-Feb,SOUSSE,KALAA KBIRA,55151.072,38.0,1451.344
3,GOULETTE,SSP SUPER SANS PLOMB,GAMMARTH,18-Feb,TUNIS,GAMMARTH,134974.992,93.0,1451.344
4,GOULETTE,GASOIL,BEN AROUS MC 34,18-Feb,BEN AROUS,BEN AROUS,96112.079,89.0,1079.911


# 1-Data Prep

### 1.1 Remove negative values

In [16]:
dataset = dataset.assign(
    prixHT=lambda x: x.apply(
        lambda row: 0 if row['SUM(MNTHT)'] < 0 or row['SUM(QTEPRD)'] == 0 else row['SUM(MNTHT)'] / row['SUM(QTEPRD)'], 
        axis=1
    )
)

In [19]:
dataset=dataset[dataset['SUM(MNTHT)']>=0]


In [20]:
dataset=dataset[dataset['SUM(QTEPRD)']>=0]


### 1.2 Null values

In [18]:
dataset.isnull().values.any()


False

### 1.3 Duplicates Values

In [24]:
dataset=dataset.drop_duplicates()
print("Shape of dataset after removing duplicates:",dataset.shape)

Shape of dataset after removing duplicates: (21746, 9)


### 1.4 Data vizualisation